In [1]:
import keras
import tensorflow as tf
#tf.keras.backend.set_floatx('float64')
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np



import pickle as pkl

f = '/Users/pres026/ml-les-microphysics/TrainingGeneration/kessler/kessler_trianing.pkl' 
with open(f, 'rb') as f:
    data = pkl.load(f)

Using TensorFlow backend.


In [2]:
print(data.keys())

dict_keys(['T_input', 'qv_input', 'qc_input', 'qr_input', 'rho_input', 'exner_input', 'T_output', 'qv_output', 'qc_output', 'qr_output', 'rho_output', 'exner_output'])


In [3]:
x = np.vstack((data['T_input'],np.log(data['qv_input']+1e-11),np.log(data['qc_input']+1e-11),np.log(data['qr_input']+1e-11), data['rho_input'], data['exner_input'])).T
y = np.vstack((data['T_output'],np.log(data['qv_output']+1e-11),np.log(data['qc_output']+1e-11),np.log(data['qr_output']+1e-11))).T


print(data['qr_output'])


scaler_x = StandardScaler()
scaler_x.fit(x)
scaler_y = StandardScaler()
scaler_y.fit(y)





xscale=scaler_x.transform(x)
yscale=scaler_y.transform(y)

[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 7.46563771e-07
 4.24955768e-08 3.79578543e-06]


In [4]:
X_train, X_test, y_train, y_test = train_test_split(xscale, yscale)

In [5]:
model = Sequential()
model.add(Dense(6, input_dim=6, kernel_initializer='normal', activation='relu'))
#model.add(Dense(64, kernel_initializer='normal', activation='relu'))
#model.add(Dense(64, kernel_initializer='normal', activation='relu'))
#model.add(Dense(64, kernel_initializer='normal', activation='relu'))
#model.add(Dense(64, kernel_initializer='normal', activation='relu'))
model.add(Dense(128, kernel_initializer='normal', activation='relu'))
model.add(Dense(128, kernel_initializer='normal', activation='relu'))
#model.add(Dense(128, kernel_initializer='normal', activation='relu',kernel_regularizer=keras.regularizers.l1(0.01)))
model.add(Dense(4, activation='linear'))
model.summary()

AttributeError: module 'tensorflow.python.framework.ops' has no attribute '_TensorLike'

In [ ]:
model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])
history = model.fit(X_train, y_train, epochs=24, batch_size=256,  verbose=1, validation_split=0.2)

In [ ]:
model = keras.models.load_model('saved_model.pb')



y_predict = model.predict(X_test) 
y_rescale_predict = scaler_y.inverse_transform(y_predict)
y_rescale_test = scaler_y.inverse_transform(y_test)

In [ ]:
import pylab as plt
plt.figure(1,figsize=(16,4))
plt.subplot(141)
plt.scatter(y_rescale_test[:,0],y_rescale_predict[:,0])
plt.scatter(y_rescale_test[:,0],y_rescale_test[:,0])
plt.grid()
plt.subplot(142)
plt.loglog(np.exp(y_rescale_test[:,1])-1e-11,np.exp(y_rescale_predict[:,1])-1e-11, '.')
plt.loglog(np.exp(y_rescale_test[:,1])-1e-11,np.exp(y_rescale_test[:,1])-1e-11, '.')
plt.grid()
plt.subplot(143)
plt.loglog(np.exp(y_rescale_test[:,2])-1e-11,np.exp(y_rescale_predict[:,2])-1e-11, '.')
plt.loglog(np.exp(y_rescale_test[:,2])-1e-11,np.exp(y_rescale_test[:,2])-1e-11, '.')
#plt.plot(y_rescale_test[:,2],y_rescale_test[:,2], '.')
plt.grid()
plt.subplot(144)
plt.loglog(np.exp(y_rescale_test[:,3])-1e-11,np.exp(y_rescale_predict[:,3])-1e-11, '.')
plt.loglog(np.exp(y_rescale_test[:,3])-1e-11,np.exp(y_rescale_test[:,3])-1e-11, '.')
plt.grid()

In [ ]:
plt.hist(data['qv_input'] + data['qr_input'] + data['qc_input'] - (data['qv_output'] + data['qr_output'] + data['qc_output']))


In [ ]:
model.save('kessler_dnn.h5')

In [ ]:
mod2 = keras.models.load_model('kessler_dnn.h5')

In [ ]:
from joblib import dump, load
dump(scaler_x, 'scaler_x.bin', compress=True)
dump(scaler_y, 'scaler_y.bin', compress=True)

In [ ]:
from autokeras import StructuredDataRegressor

In [ ]:
search = StructuredDataRegressor(max_trials=150, loss='mean_absolute_error')
search.fit(x=X_train, y=y_train, verbose=1)

In [ ]:
model = search.export_model()
model.summary()

y_predict = model.predict(X_test) 
y_rescale_predict = scaler_y.inverse_transform(y_predict)
y_rescale_test = scaler_y.inverse_transform(y_test)

import pylab as plt
plt.figure(1,figsize=(16,4))
plt.subplot(141)
plt.scatter(y_rescale_test[:,0],y_rescale_predict[:,0])
plt.scatter(y_rescale_test[:,0],y_rescale_test[:,0])
plt.grid()
plt.subplot(142)
plt.loglog(np.exp(y_rescale_test[:,1])-1e-11,np.exp(y_rescale_predict[:,1])-1e-11, '.')
plt.loglog(np.exp(y_rescale_test[:,1])-1e-11,np.exp(y_rescale_test[:,1])-1e-11, '.')
plt.grid()
plt.subplot(143)
plt.loglog(np.exp(y_rescale_test[:,2])-1e-11,np.exp(y_rescale_predict[:,2])-1e-11, '.')
plt.loglog(np.exp(y_rescale_test[:,2])-1e-11,np.exp(y_rescale_test[:,2])-1e-11, '.')
#plt.plot(y_rescale_test[:,2],y_rescale_test[:,2], '.')
plt.grid()
plt.subplot(144)
plt.loglog(np.exp(y_rescale_test[:,3])-1e-11,np.exp(y_rescale_predict[:,3])-1e-11, '.')
plt.loglog(np.exp(y_rescale_test[:,3])-1e-11,np.exp(y_rescale_test[:,3])-1e-11, '.')
plt.grid()

In [ ]:
model.save('kessler_dnn.h5')